In [1]:
from spectra import Spectra
import numpy as np
import matplotlib.pyplot as plt

/home/rodrigo/anaconda3/envs/py39/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
## simulation parameters

#previously used arguments will be saved for records

#create_simulated_dataset("Unsup","_unsup_2_5",16,5)
#Mean SNR (Water Unsup) is 2.575164358578593 and Std Dev is 0.6913887125670769

#create_simulated_dataset("Unsup","_unsup_5",7.5,3)
#Mean SNR (Water Unsup) is 4.9142298415942465 and Std Dev is 1.6535384608901584

#create_simulated_dataset("Unsup","_unsup_10",3.5,1)
#Mean SNR (Water Unsup) is 9.799043876447383 and Std Dev is 2.8530347175303623

#create_simulated_dataset("Sup","_sup_2_5",16,5)
#Mean SNR (Water Unsup) is 2.2931368924596245 and Std Dev is 0.543460556970006

#create_simulated_dataset("Sup","_sup_5",7,3)
#Mean SNR (Water Unsup) is 4.909138679775567 and Std Dev is 1.4432091980587023

#create_simulated_dataset("Sup","_sup_10",3.1,1)
#Mean SNR (Water Unsup) is 10.055709761306701 and Std Dev is 2.4580875967655844

Mean SNR (Water Unsup) is 10.055709761306701 and Std Dev is 2.4580875967655844


In [3]:
def create_simulated_dataset(in_suffix,out_suffix,amp_base,amp_var):

    prefix = "fpc_sim_data/"
    out_prefix="noisy_datasets"

    num_scans, num_trans = 250, 160

    spec = Spectra()
    spec.load_from_ground_truth_csvs(f"{prefix}fidsON_{in_suffix}.csv", f"{prefix}fidsOFF_{in_suffix}.csv", f"{prefix}ppm_{in_suffix}.csv", f"{prefix}t_{in_suffix}.csv")
    spec.select_scans(0, num_scans)
    spec.make_transients(transient_count=num_trans*2)

    rfn_base,  rfn_var = 10, 10
    lfn_base, lfn_var = 7, 3
    rpn_base, rpn_var = 60, 30
    ran_base, ran_var = amp_base, amp_var

    spec.add_random_amplitude_noise(noise_level_base=ran_base, noise_level_scan_var=ran_var)
    spec.add_random_frequency_noise(noise_level_base=rfn_base, noise_level_scan_var=rfn_var)
    spec.add_linear_frequency_noise(offset_var=lfn_base, slope_var=lfn_var)
    spec.add_random_phase_noise(noise_level_base=rpn_base, noise_level_scan_var=rpn_var)
    ###################################################################################################################
    # Verification of Results (SNR, FPC Range)
    ###################################################################################################################
    new_ppm = spec.ppm[0, :]
    new_ppm = np.squeeze(np.ndarray.round(new_ppm, 2))
    Cr_indClose, Cr_indFar = np.amax(np.where(new_ppm == 2.8)), np.amin(np.where(new_ppm == 3.2))
    dt_indClose, dt_indFar = np.amax(np.where(new_ppm == 10.0)), np.amin(np.where(new_ppm == 10.8))
    old_on_scans = spec.specs
    Cr_SNR, Cr_SNR1 = np.zeros((num_scans, num_trans)), np.zeros((num_scans, num_trans))

    for k in range(num_scans):
        for j in range(num_trans):
            on_specs = old_on_scans[k, :, 1, j]
            on_specs = np.squeeze(on_specs)
            # Amp of peak
            max_peak = np.amax(on_specs[Cr_indFar:Cr_indClose])
            # Std Dev of Noise
            dt = np.polyfit(new_ppm[dt_indFar:dt_indClose], on_specs[dt_indFar:dt_indClose], 2)
            
            sizeFreq = new_ppm[dt_indFar:dt_indClose].shape[0]
            stdev_Man = np.sqrt(np.sum(np.square(np.real(on_specs[dt_indFar:dt_indClose] - np.polyval(dt, new_ppm[dt_indFar:dt_indClose])))) / (sizeFreq - 1))
            Cr_SNR[k, j] = np.real(max_peak) / (2 * stdev_Man)
            
            #print(f"SNR of (Water Unsup) scan {k} transient {j} is {Cr_SNR[k, j]}")
            #print(f"SNR of (Water Sup) scan {k} transient {j} is {Cr_SNR1[k, j]}")

    print(f"Mean SNR (Water Unsup) is {np.mean(Cr_SNR)} and Std Dev is {np.std(Cr_SNR)}")

    ###################################################################################################################
    # Saving Results
    ###################################################################################################################
    x_data = spec.specs
    y_data_freq = -spec.added_noise["frequency_drift"]
    y_data_phase = -spec.added_noise["phase_drift"]
    off_fids = spec.fids[:, :, 0, :]
    on_fids = spec.fids[:, :, 1, :]
    ppm = spec.ppm[0].flatten()
    t = spec.t[0].flatten()

    np.save(f"{out_prefix}/x_data{out_suffix}.npy", x_data)
    np.save(f"{out_prefix}/y_dataFreq{out_suffix}.npy", y_data_freq)
    np.save(f"{out_prefix}/y_dataPhase{out_suffix}.npy", y_data_phase)
    np.save(f"{out_prefix}/off_fids{out_suffix}.npy", off_fids)
    np.save(f"{out_prefix}/on_fids{out_suffix}.npy", on_fids)
    np.save(f"{out_prefix}/ppm{out_suffix}.npy",ppm)
    np.save(f"{out_prefix}/t{out_suffix}.npy",t)

